# Проект. Проблема дорожно-транспортных происшествий в России. 
# Исследование данных и изучение взаимосвязей. С чем связана аварийность на дорогах.
## На примере Кировской области

## 1. Выполним предобработку для удобства работы с данными: выбрать оптимальные названия столбцов и типы данных. Понадобится дать ответы на вопросы заказчика.

### 1.1  Вопросы заказчика
- Как распределено количество участников ДТП и почему? Встречаются ли аномальные значения или выбросы? Если да, то с чем они могут быть связаны? Для числа участников найдите наиболее типичное значение. По желанию можете проверить распределения и других столбцов.

- Предположите, между какими столбцами в датасете Kirovskaya_oblast.csv высокая корреляция. Проверьте своё предположение.

- Как связаны категории аварий и погодные условия?

- Постройте процентную разбивку аварий по видам освещённости. При этом учитывайте пол участника и сделайте расчёты для мужчин и женщин отдельно.

- Исследуйте, чем отличаются аварии без пострадавших от тех, в которых был один пострадавший или более.

- Сделайте общий вывод о связи аварийности с другими факторами. Какие рекомендации вы можете дать заказчику?

## 2.  Описание данных
Датасет `Kirovskaya_oblast.csv` содержит информацию о ДТП в Кировской области:

- geometry.coordinates — координаты ДТП;
- id — идентификатор ДТП;
- properties.tags — тег происшествия;
- properties.light — освещённость;
- properties.point.lat — широта;
- properties.point.long — долгота;
- properties.nearby — ближайшие объекты;
- properties.region — регион;
- properties.scheme — схема ДТП;
- properties.address — ближайший адрес;
- properties.weather — погода;
- properties.category — категория ДТП;
- properties.datetime — дата и время ДТП;
- properties.injured_count — число пострадавших;
- properties.parent_region — область;
- properties.road_conditions — состояние покрытия;
- properties.participants_count — число участников;
- properties.participant_categories — категории участников.

Датасет `Kirovskaya_oblast_participiants.csv` хранит сведения об участниках:
- role — роль;
- gender — пол;
- violations — какие правила дорожного движения были нарушены конкретным участником;
- health_status — состояние здоровья после ДТП;
- years_of_driving_experience — число лет опыта;
- id — идентификатор ДТП.

## 3. Загрузим данные и выполним небольшую предобработку

## 4.  Исследование данных

- 3.1 Как распределено количество участников ДТП и почему? Встречаются ли аномальные значения или выбросы? Если да, то с чем они могут быть связаны? Для числа участников найдите наиболее типичное значение. По желанию можете проверить распределения и других столбцов.
​
- 3.2 Предположите, между какими столбцами в данных высокая корреляция? Проверьте своё предположение.
​
- 3.3 Как связаны категории аварий и погодные условия?
​
- 3.4 Постройте процентную разбивку аварий по видам освещённости. При этом учитывайте пол участника и сделайте расчёты для мужчин и женщин отдельно.
​
- 3.5 Исследуйте, чем отличаются аварии без пострадавших от тех, в которых был один пострадавший или более.
​
- 3.6 Сделайте общий вывод о связи аварийности с другими факторами. Какие рекомендации вы можете дать заказчику?
​


In [30]:
# Импортируем библиотеки
import pandas as pd

# Загружаем библиотеки для визуализации данных
import matplotlib.pyplot as plt
import seaborn as sns

# Загружаем библиотеку для расчёта коэффициента корреляции phi_k
from phik import phik_matrix

df_info = pd.read_csv('https://code.s3.yandex.net/datasets/Kirovskaya_oblast.csv')
df_partic = pd.read_csv('https://code.s3.yandex.net/datasets/Kirovskaya_oblast_participiants.csv')

## Для удобства изменим названия столбцов первого датафрейма, используя код ниже:
df_info.columns = [c.replace('properties.', '') for c in df_info.columns]

## Знакомимся с данными датасета


In [31]:
# Выведем первые строки методом head(), а информацию о датафрейме методом info():
df_info.info()
df_info.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14517 entries, 0 to 14516
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   geometry.coordinates    14517 non-null  object 
 1   id                      14517 non-null  int64  
 2   tags                    14517 non-null  object 
 3   light                   14517 non-null  object 
 4   point.lat               14485 non-null  float64
 5   point.long              14485 non-null  float64
 6   nearby                  14517 non-null  object 
 7   region                  14517 non-null  object 
 8   scheme                  13380 non-null  float64
 9   address                 13843 non-null  object 
 10  weather                 14517 non-null  object 
 11  category                14517 non-null  object 
 12  datetime                14517 non-null  object 
 13  injured_count           14517 non-null  int64  
 14  parent_region           14517 non-null

,geometry.coordinates,id,tags,light,point.lat,point.long,nearby,region,scheme,address,weather,category,datetime,injured_count,parent_region,road_conditions,participants_count,participant_categories
0,"[47.875603, 57.24379]",1983180,Дорожно-транспортные происшествия,Светлое время суток,57.243790,47.875603,[],Яранский район,600.0,Р-176 Вятка Чебоксары - Йошкар-Ола - Киров - С...,['Дождь'],Опрокидывание,2017-07-01 18:00:00,1,Кировская область,['Мокрое'],3,['Все участники']
1,"[47.87903, 57.304807]",2889433,Дорожно-транспортные происшествия,Светлое время суток,57.304807,47.879030,"['Административные здания', 'Нерегулируемый пе...",Яранский район,710.0,"г Яранск, ул Кирова, 10",['Ясно'],Наезд на пешехода,2023-09-12 17:10:00,1,Кировская область,"['Сухое', 'Отсутствие, плохая различимость гор...",2,"['Все участники', 'Пешеходы']"
2,"[47.840781, 57.297156]",2591208,Дорожно-транспортные происшествия,Сумерки,57.297156,47.840781,"['Жилые дома индивидуальной застройки', 'Нерег...",Яранский район,NaN,"г Яранск, ул Чапаева, 80",['Пасмурно'],Съезд с дороги,2021-07-02 21:25:00,1,Кировская область,['Мокрое'],1,['Все участники']
3,"[47.834365, 57.244775]",2577639,Дорожно-транспортные происшествия,Светлое время суток,57.244775,47.834365,['Жилые дома индивидуальной застройки'],Яранский район,200.0,"м Знаменка, ул Кирова, 15",['Пасмурно'],Столкновение,2021-05-31 18:55:00,1,Кировская область,['Сухое'],2,"['Все участники', 'Мотоциклисты']"
4,"[47.968197, 57.357738]",1981026,Дорожно-транспортные происшествия,Светлое время суток,57.357738,47.968197,['Нерегулируемый перекрёсток неравнозначных ул...,Яранский район,NaN,"с/п Никольское, Киров-Советск- Яранск - подъез...",['Ясно'],Опрокидывание,2018-05-16 16:25:00,2,Кировская область,"['Сухое', 'Отсутствие, плохая различимость гор...",2,['Все участники']


### Предварительные выводы после первичного анализа данных

Датасет `df_info` содержит 18 столбцов и 14517 строк с информацией о ДТП в Кировской области.

Изучим типы данных и их корректность:
- **Булевые значения (bool).** Данные типа `bool` отсутствуют.
- **Целочисленные значения (int64).** Данные типа `int64` содержатся в полях `id`, `injured_count` и `participants_count`. Корректно. 
Но целесообразно оптимизировать размерность в полях `injured_count` и `participants_count`.
- **Числовые значения с плавающей запятой (float64).** 3 столбца с типом `float64`:
    - `point.lat`, 
    - `point.long`, 
    - `scheme`
    содержат координаты и схему ДТП соответственно, и представлены типом `float64`, что корректно, т.к. данные могут иметь дробную часть. 
    Стоит уточнить какого рода данные хранятся в `scheme`.
- **Строковые данные (object).** 12 столбцов имеют тип данных `object`. Стоит преобразовать столбец `datetime`, приведя его к типу даты/времени. Остальные - корректно.

- В результате анализа типов данных можем сделать вывод, что большинство столбцов представлены корректно. 
Целесообразно оптимизировать размерность данных в полях `injured_count` и `participants_count`. Также целесообразно преобразовать столбец `datetime`, приведя их к типу даты/времени. Также обратим внимание на  некоторую неоднородность стиля названий колонок. Стоит привести все названия к стилю `snake_case`, убрать лишние пробелы и точки.

Пропуски содержатся только в столбцах `point.lat`, `point.long`, `scheme`, `address`. Также проверим и другие столбцы: в них могут встречаться значения-индикаторы, которыми заменены пропуски.
Судя по первому знакомству с данными, значения в столбцах соответствуют своему описанию.

In [32]:
# Выведем первые строки методом head(), а информацию о датафрейме методом info():
df_partic.info()
df_partic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31235 entries, 0 to 31234
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   role                         31235 non-null  object 
 1   gender                       30387 non-null  object 
 2   violations                   31235 non-null  object 
 3   health_status                31135 non-null  object 
 4   years_of_driving_experience  16909 non-null  float64
 5   id                           31235 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 1.4+ MB


,role,gender,violations,health_status,years_of_driving_experience,id
0,Водитель,Мужской,['Несоответствие скорости конкретным условиям ...,"Раненый, находящийся (находившийся) на амбулат...",26.0,1983180
1,Водитель,Мужской,[],Не пострадал,34.0,2889433
2,Пассажир,Мужской,[],"Раненый, находящийся (находившийся) на амбула...",NaN,2591208
3,Пассажир,Мужской,[],"Раненый, находящийся (находившийся) на амбула...",NaN,2591208
4,Водитель,Мужской,[],Не пострадал,27.0,2577639


### Предварительные выводы после первичного анализа данных

Датасет `df_partic` содержит 6 столбцов и 31235 строк с информацией об участниках ДТП в Кировской области.

Изучим типы данных и их корректность:
- **Булевые значения (bool).** Данные типа `bool` отсутствуют.
- **Целочисленные значения (int64).** Данные типа `int64` содержатся в поле `id`. Корректно. 
- **Числовые значения с плавающей запятой (float64).** 1 столбец с типом `float64`: `years_of_driving_experience`. Корректно. 
- **Строковые данные (object).** 4 столбца имеют тип данных `object`. Корректно.

- В результате анализа типов данных можем сделать вывод, что все столбцы представлены корректно. 
Есть единообразие в стиле - `snake_case`, регистр нижний.

Пропуски содержатся только в столбцах `gender` и `years_of_driving_experience`. Также проверим и другие столбцы: в них могут встречаться значения-индикаторы, которыми заменены пропуски.
Судя по первому знакомству с данными, значения в столбцах соответствуют своему описанию.

### Подготовка единого датафрейма

Соединим данные о ДТП `df_info` с даными об участниках ДТП `df_partic`.

Каждая строка в этих датафреймах — это информация о каждом ДТП и его участниках, поэтому такие данные удобно соединить в один датафрейм, который можно использовать для поиска закономерностей в данных. Соединять данные будем по идентификатору `id`.

Соединим данные, используя значение параметра `how` — `left`.

In [33]:
# Соединяем данные в единый датафрейм df
df = df_info.merge(df_partic, on='id', how='left')

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32412 entries, 0 to 32411
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   geometry.coordinates         32412 non-null  object 
 1   id                           32412 non-null  int64  
 2   tags                         32412 non-null  object 
 3   light                        32412 non-null  object 
 4   point.lat                    32362 non-null  float64
 5   point.long                   32362 non-null  float64
 6   nearby                       32412 non-null  object 
 7   region                       32412 non-null  object 
 8   scheme                       29821 non-null  float64
 9   address                      30931 non-null  object 
 10  weather                      32412 non-null  object 
 11  category                     32412 non-null  object 
 12  datetime                     32412 non-null  object 
 13  injured_count   

Данные соединены, и информация обо всех ДТП сохранилась.

---

## 2. Предобработка данных

### Подготовим данные к исследовательскому анализу:

### 2.1. Оптимизируем типы данных

С данными работы по оптимизации типов не так много — понадобится только провести оптимизацию целочисленных данных.

Для оптимизации будем использовать метод `pd.to_numeric` с аргументом `downcast='integer'`:

In [35]:
# Оптимизируем целочисленный тип данных в датафрейме df

df['injured_count'] = pd.to_numeric(df['injured_count'], downcast='integer')
df['participants_count'] = pd.to_numeric(df['participants_count'], downcast='integer')

In [36]:
# Проверим типы данных в датафрейме df с помощью атрибута dtypes
df.dtypes

geometry.coordinates            object
id                               int64
tags                            object
light                           object
point.lat                      float64
point.long                     float64
nearby                          object
region                          object
scheme                         float64
address                         object
weather                         object
category                        object
datetime                        object
injured_count                     int8
parent_region                   object
road_conditions                 object
participants_count                int8
participant_categories          object
role                            object
gender                          object
violations                      object
health_status                   object
years_of_driving_experience    float64
dtype: object

Типы данных были успешно оптимизированы с понижением размерности целочисленных данных.

---

### 2.2. Проверяем наличие пропусков

При первичном анализе мы обнаружили пропуски в столбце `point.lat`, `point.long`, `scheme`, `address`, `gender` и `years_of_driving_experience`. Узнаем абсолютное и относительное количество пропусков в этих столбцах.

In [37]:
# Применяем метод isna() к датафрейму df
df.isna().sum()

geometry.coordinates               0
id                                 0
tags                               0
light                              0
point.lat                         50
point.long                        50
nearby                             0
region                             0
scheme                          2591
address                         1481
weather                            0
category                           0
datetime                           0
injured_count                      0
parent_region                      0
road_conditions                    0
participants_count                 0
participant_categories             0
role                            1177
gender                          2025
violations                      1177
health_status                   1277
years_of_driving_experience    15503
dtype: int64

In [38]:
# Подсчитываем долю строк с пропусками в процентах
round((df.isna().sum() / df.shape[0])*100, 2)

geometry.coordinates            0.00
id                              0.00
tags                            0.00
light                           0.00
point.lat                       0.15
point.long                      0.15
nearby                          0.00
region                          0.00
scheme                          7.99
address                         4.57
weather                         0.00
category                        0.00
datetime                        0.00
injured_count                   0.00
parent_region                   0.00
road_conditions                 0.00
participants_count              0.00
participant_categories          0.00
role                            3.63
gender                          6.25
violations                      3.63
health_status                   3.94
years_of_driving_experience    47.83
dtype: float64

В датафрейме `df` обнаружены пропуски в 9-ти столбцах:
- 15503 пропуска в столбце `years_of_driving_experience` — это 47% данных. Количество пропусков достаточно большое, чтобы их просто удалить.
- В столбцах `gender`, `health_status` и `years_of_driving_experience` незначительное количество пропусков (от 3,63% до 7,99% данных). Данных менее 10%, можно удалить пропуски в этих столбцах. Пропуски в `scheme`, `address`, `role` оставли как есть.
- Наименьшее количество пропусков в столбцах `point.lat` и `point.long`: по 50 пропусков - это 0,15% данных. Также можно их удалить.

Пропусков в столбце `years_of_driving_experience` большое количество - удалять и заменять некорректно. Пропуски в остальных столбцах удаляем.

In [39]:
df = df.dropna(subset=['point.lat', 'point.long', 'gender', 'violations', 'health_status'])


In [40]:
# Проверим
round((df.isna().sum() / df.shape[0])*100, 2)

geometry.coordinates            0.00
id                              0.00
tags                            0.00
light                           0.00
point.lat                       0.00
point.long                      0.00
nearby                          0.00
region                          0.00
scheme                          7.85
address                         4.50
weather                         0.00
category                        0.00
datetime                        0.00
injured_count                   0.00
parent_region                   0.00
road_conditions                 0.00
participants_count              0.00
participant_categories          0.00
role                            0.00
gender                          0.00
violations                      0.00
health_status                   0.00
years_of_driving_experience    44.96
dtype: float64

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30343 entries, 0 to 31246
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   geometry.coordinates         30343 non-null  object 
 1   id                           30343 non-null  int64  
 2   tags                         30343 non-null  object 
 3   light                        30343 non-null  object 
 4   point.lat                    30343 non-null  float64
 5   point.long                   30343 non-null  float64
 6   nearby                       30343 non-null  object 
 7   region                       30343 non-null  object 
 8   scheme                       27960 non-null  float64
 9   address                      28977 non-null  object 
 10  weather                      30343 non-null  object 
 11  category                     30343 non-null  object 
 12  datetime                     30343 non-null  object 
 13  injured_count        

```--------Доработать----------
Причинами появления пропусков могут быть следующие факторы:
- возможные технические ошибки общего характера, например при парсинге, объединении таблиц и пр.
- если данные собираются из разных источников, данные могут отсутствовать в некоторых источниках, например, у некоторых заведений может не быть фиксированного `middle_avg_bill`
- отсутствие данных `price` может объясняться отсутствием четкой ценовой категории
- на отсутствие определенных данных может влиять специфика работы заведений, например, `middle_coffee_cup` актуально для кофеен, но в заведениях, где кофе не является основным товаром, эта информация будет отсутствовать
- причиной отсутствия данных по количеству мест `seats` может быть непостоянное их количество в определенных заведедниях или их отсутсвие, например, в булочных с продажей только навынос
--------Доработать----------```

### 2.3. Явные и неявные дубликаты в данных

Проверим данные на наличие явных и неявных дубликатов.

In [42]:
# Проверяем полные дубликаты в датафрейме df
df.duplicated().sum()

np.int64(9428)

В датафрейме 9428 полных дубликатов строк.

In [52]:
# Проверяем неявные дубликаты в датафрейме df
df.duplicated(subset='id').sum()

np.int64(17323)

In [ ]:
# Проверяем уникальные значения в категориальных столбцах
for column in ['tags', 'light', 'gender', 'role']:
    print(f'Уникальные значения в столбце {column}:')
    print(df[column].sort_values().unique())
    print()

Уникальные значения в столбце tags:
['Дорожно-транспортные происшествия']

Уникальные значения в столбце light:
['В темное время суток, освещение включено'
 'В темное время суток, освещение не включено'
 'В темное время суток, освещение отсутствует' 'Светлое время суток'
 'Сумерки']

Уникальные значения в столбце gender:
['Женский' 'Мужской']

Уникальные значения в столбце role:
['Велосипедист' 'Водитель' 'Пассажир' 'Пешеход'
 'Пешеход, перед ДТП находившийся в (на) ТС в качестве водителя или пешеход, перед ДТП находившийся в (на) ТС в качестве пассажира']



In [61]:
# В новой переменной указываем, по каким столбцам искать дубликаты
duplicates = df[df.duplicated(subset=['id', 'address', 'tags', 'light', 'gender', 'role', 'violations', 'health_status'], keep=False)]

# Выводим результат
display(duplicates.head(40))

,geometry.coordinates,id,tags,light,point.lat,point.long,nearby,region,scheme,address,...,injured_count,parent_region,road_conditions,participants_count,participant_categories,role,gender,violations,health_status,years_of_driving_experience
3,"[47.840781, 57.297156]",2591208,Дорожно-транспортные происшествия,Сумерки,57.297156,47.840781,"['Жилые дома индивидуальной застройки', 'Нерег...",Яранский район,NaN,"г Яранск, ул Чапаева, 80",...,1,Кировская область,['Мокрое'],1,['Все участники'],Пассажир,Мужской,[],"Раненый, находящийся (находившийся) на амбула...",NaN
4,"[47.840781, 57.297156]",2591208,Дорожно-транспортные происшествия,Сумерки,57.297156,47.840781,"['Жилые дома индивидуальной застройки', 'Нерег...",Яранский район,NaN,"г Яранск, ул Чапаева, 80",...,1,Кировская область,['Мокрое'],1,['Все участники'],Пассажир,Мужской,[],"Раненый, находящийся (находившийся) на амбула...",NaN
5,"[47.840781, 57.297156]",2591208,Дорожно-транспортные происшествия,Сумерки,57.297156,47.840781,"['Жилые дома индивидуальной застройки', 'Нерег...",Яранский район,NaN,"г Яранск, ул Чапаева, 80",...,1,Кировская область,['Мокрое'],1,['Все участники'],Водитель,Мужской,['Другие нарушения ПДД водителем'],"Раненый, находящийся (находившийся) на амбула...",2.0
6,"[47.840781, 57.297156]",2591208,Дорожно-транспортные происшествия,Сумерки,57.297156,47.840781,"['Жилые дома индивидуальной застройки', 'Нерег...",Яранский район,NaN,"г Яранск, ул Чапаева, 80",...,1,Кировская область,['Мокрое'],1,['Все участники'],Водитель,Мужской,['Другие нарушения ПДД водителем'],"Раненый, находящийся (находившийся) на амбула...",2.0
11,"[47.9089, 57.3225]",1986895,Дорожно-транспортные происшествия,"В темное время суток, освещение отсутствует",57.322500,47.908900,[],Яранский район,840.0,"Киров-Советск- Яранск - подъезд к г. Яранск, 5 км",...,1,Кировская область,"['Сухое', 'Отсутствие, плохая различимость гор...",2,"['Все участники', 'Пешеходы']",Пассажир,Женский,[],Получил телесные повреждения с показанием к ле...,NaN
12,"[47.9089, 57.3225]",1986895,Дорожно-транспортные происшествия,"В темное время суток, освещение отсутствует",57.322500,47.908900,[],Яранский район,840.0,"Киров-Советск- Яранск - подъезд к г. Яранск, 5 км",...,1,Кировская область,"['Сухое', 'Отсутствие, плохая различимость гор...",2,"['Все участники', 'Пешеходы']",Пассажир,Женский,[],Получил телесные повреждения с показанием к ле...,NaN
13,"[47.9089, 57.3225]",1986895,Дорожно-транспортные происшествия,"В темное время суток, освещение отсутствует",57.322500,47.908900,[],Яранский район,840.0,"Киров-Советск- Яранск - подъезд к г. Яранск, 5 км",...,1,Кировская область,"['Сухое', 'Отсутствие, плохая различимость гор...",2,"['Все участники', 'Пешеходы']",Водитель,Женский,[],Не пострадал,16.0
14,"[47.9089, 57.3225]",1986895,Дорожно-транспортные происшествия,"В темное время суток, освещение отсутствует",57.322500,47.908900,[],Яранский район,840.0,"Киров-Советск- Яранск - подъезд к г. Яранск, 5 км",...,1,Кировская область,"['Сухое', 'Отсутствие, плохая различимость гор...",2,"['Все участники', 'Пешеходы']",Водитель,Женский,[],Не пострадал,16.0
15,"[47.895906, 57.376484]",2600500,Дорожно-транспортные происшествия,Светлое время суток,57.376484,47.895906,[],Яранский район,190.0,"с/п Опытнопольское, Р-176 Вятка Чебоксары - Йо...",...,2,Кировская область,['Сухое'],4,['Все участники'],Пассажир,Мужской,[],Получил телесные повреждения с показанием к ле...,NaN
16,"[47.895906, 57.376484]",2600500,Дорожно-транспортные происшествия,Светлое время суток,57.376484,47.895906,[],Яранский район,190.0,"с/п Опытнопольское, Р-176 Вятка Чебоксары - Йо...",...,2,Кировская область,['Сухое'],4,['Все участники'],Пассажир,Мужской,[],Получил телесные повреждения с показанием к ле...,NaN
